## Linking in Spark


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_dev/docs/demos/examples/spark/deduplicate_1k_synthetic.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_dev
# !pip install pyspark

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

from splink.backends.spark import similarity_jar_location

conf = SparkConf()
# This parallelism setting is only suitable for a small toy example
conf.set("spark.driver.memory", "12g")
conf.set("spark.default.parallelism", "8")


# Add custom similarity functions, which are bundled with Splink
# documented here: https://github.com/moj-analytical-services/splink_scalaudfs
path = similarity_jar_location()
conf.set("spark.jars", path)

sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)
spark.sparkContext.setCheckpointDir("./tmp_checkpoints")

24/06/22 10:55:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Disable warnings for pyspark - you don't need to include this
import warnings

spark.sparkContext.setLogLevel("ERROR")
warnings.simplefilter("ignore", UserWarning)

In [4]:
from splink import splink_datasets

pandas_df = splink_datasets.fake_1000

df = spark.createDataFrame(pandas_df)

In [5]:
import splink.comparison_library as cl
from splink import Linker, SettingsCreator, SparkAPI, block_on

settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
        cl.NameComparison("first_name"),
        cl.NameComparison("surname"),
        cl.DateOfBirthComparison(
            "dob",
            input_is_string=True,
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        cl.EmailComparison("email"),
    ],
    blocking_rules_to_generate_predictions=[
        block_on("first_name"),
        "l.surname = r.surname",  # alternatively, you can write BRs in their SQL form
    ],
    retain_intermediate_calculation_columns=True,
    em_convergence=0.01,
)

In [6]:
linker = Linker(df, settings, database_api=SparkAPI(spark_session=spark))
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email",
]

linker.training.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)

Probability two random records match is estimated to be  0.0806.                
This means that amongst all possible pairwise record comparisons, one in 12.41 are expected to match.  With 499,500 total possible comparisons, we expect a total of around 40,246.67 matching pairs


In [7]:
linker.training.estimate_u_using_random_sampling(max_pairs=5e5)

----- Estimating u probabilities using random sampling -----
                                                                                
Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [8]:
training_blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"
training_session_fname_sname = (
    linker.training.estimate_parameters_using_expectation_maximisation(training_blocking_rule)
)

training_blocking_rule = "l.dob = r.dob"
training_session_dob = linker.training.estimate_parameters_using_expectation_maximisation(
    training_blocking_rule
)

                                                                                
----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.first_name = r.first_name and l.surname = r.surname

Parameter estimates will be made for the following comparison(s):
    - dob
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname
                                                                                
Iteration 1: Largest change in params was -0.706 in probability_two_random_records_match
Iteration 2: Largest change in params was 0.0583 in the m_probability of email, level `All other comparisons`
Iteration 3: Largest change in params was 0.0217 in the m_probability of email, level `All other comparisons`
Iteration 4: Largest change in params was 0.00941 in the m_probability of email, level `All other comparisons`

EM converged after

In [12]:
results = linker.inference.predict(threshold_match_probability=0.9)

In [11]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,tf_first_name_r,bf_first_name,...,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,tf_email_l,tf_email_r,bf_email,bf_tf_adj_email,match_key
0,14.992986,0.999969,192,194,Isla,Isla,4,0.009,0.009,11.371023,...,0.626291,1.000000,iprice@carr.com,iprice@carr.com,4,0.004,0.004,8.473825,11.429930,0
1,10.461631,0.999291,303,304,Charlie,Charlie,4,0.006,0.006,11.371023,...,5.870654,2.159371,candrews15@thompson.net,NaN,0,0.002,0.211,0.349824,1.000000,0
2,15.283680,0.999975,79,84,Ryan,Ryan,4,0.005,0.005,11.371023,...,0.626291,1.000000,r.cole1@ramirez-anthony.com,r.cole1@ramtrez-anihony.com,3,0.005,0.001,210.647668,1.000000,0
3,7.419792,0.994194,111,113,Oliver,Oliver,4,0.028,0.028,11.371023,...,5.870654,0.411903,NaN,NaN,4,0.211,0.211,8.473825,0.216681,0
4,4.030262,0.942327,220,222,Logan,Logan,4,0.010,0.010,11.371023,...,0.626291,1.000000,l.feruson46@sahh.com,l.ferguson46@shah.com,2,0.001,0.002,252.479846,1.000000,0
